# Predictions

In this notebook you could test a pretrained model with real sentences

In [ ]:
import numpy as np
import os
import pandas as pd

import pickle
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.models import model_from_json

# Only for testing in local
import sample

In [ ]:
# Import and use intentbasedbot
def get_model_and_encoder(lang):
	base_path = 'intentbasedbot'
	print('Loading model and encoder')
	with open(os.path.join(base_path, 'model', f'model_{lang}.json'), 'r') as json_file:
		model = model_from_json(json_file.read())
	print('Model definition loaded from disk')

	try:
		model.load_weights(os.path.join(base_path, 'model', f'model_{lang}.h5'))
		print("Model weights loaded from disk")
	except:
		print('Error abort prediction')
		abort(500)

	encoder = LabelEncoder()
	encoder.classes_ = np.load(os.path.join(base_path, 'model', f'classes_{lang}.npy'))
	print('Encoder definition loaded from disk')

	with open(os.path.join(base_path, 'model', f'tokenizer_{lang}.pickle'), 'rb') as handle:
		tokenizer = pickle.load(handle)

	return model, encoder, tokenizer

In [ ]:
def preprocessing(sentences, tokenizer, max_len = 100):
	"""
		:sentences: List of strings
		:returns: (1,100) numpy array
	"""
	sequences = tokenizer.texts_to_sequences(sentences)

	# Transforming the list of indexes to a 2D tensor (sample, maxlen)
	return pad_sequences(sequences, maxlen=max_len)

In [ ]:
model, encoder, tokenizer = get_model_and_encoder('es')

In [ ]:
sample_sentence = sample.get_sample(10)
test_data = preprocessing(sample_sentence, tokenizer)
predictions = model.predict_classes(test_data, batch_size=10, verbose=1)
for i in range(0, len(predictions)):
    print('Class for {}: {}'.format(sample_sentence[i], encoder.inverse_transform(predictions[i])))

In [ ]:
dataset_path = 'data/snips_utterances.csv'
intent_column = 'Intention'
language = 'Spanish'

all_intents_df = pd.read_csv(dataset_path)
all_intents_df.sample(10)